#### Geoff Counihan 8/18/2017
## Intro

This is the scratchpad/notebook for a coding assignment I had during an interview process. It is my first time making a RestFUL API. If I had additional time there are a few extra additions I would add/fix. They are:
    - Add error message handling for attribute errors like total=gran or skus=a
    - Reduce the iso datetime format from "2017-10-01T00:00:00.000Z" to "2017-10-01"
    - Create better response labeling to differentiate between price and unit counts
    - Rethink logic for filters to clean up the code
    - Serve the API on Google's App Engine
    

## RESTful webservice

Part 1) 

Design a RESTful web service that will provide demand forecasts to users. This service
should support at least three types of requests:
- i) Single SKU forecasts: In this case, users provide at least a SKU number, a target week for the
forecast, and optionally additional factors affecting demand such as the presence of
promotions and price.
- ii) Batch forecasts: In this case, users specify a time window and receive baseline forecasts for
all stocked SKUs within that time window.
- iii) Manual overrides: In this case a user may provide an override to the forecast for a specific
SKU/week, the total forecasted demand for a specific SKU over a range of weeks, the total
forecasted demand over a class of SKUs for a given week, or the total forecasted demand over a
class and range of weeks.

Your answer should consist of concrete examples of the inputs (appropriately formatted
depending on whether you use GET or POST requests) and the JSON formatted responses for
each of the three types of request.

#### SKU info

Pull information about individual skus based on a sku dictionary. Populated from a product info

request:
http://0.0.0.0/api/v1.0/skus/2

response:
{
  "2": {
    "Price": "$4.50", 
    "Product Category": "fasteners", 
    "Product Description": "Red Stapler", 
    "Promotion": "nan", 
    "SKU": "2"
  }
}

In [492]:
prod_info = {
        "Product Information":{
        "1":{
        "SKU":"1",
        "Product Description":"Legal Paper, 50 pages",
        "Price":"15.00",
        "Promotion":"$13.00",
        "Product Category":"paper_goods"
        },
        "2":{
        "SKU":"2",
        "Product Description":"Red Stapler",
        "Price":"$4.50",
        "Promotion":"nan",
        "Product Category":"fasteners"
        }}}

In [495]:
float(prod_info['Product Information']['1']['Price'])

15.0

# i)

Search for specific skus based on date range

GET http://0.0.0.0/api/v1.0/skus/forecasts?wk_range=2015-01-01,2015-02-01&skus=1

{"1":{"2015-01-04T00:00:00.000Z":87,"2015-01-11T00:00:00.000Z":93,"2015-01-18T00:00:00.000Z":61,"2015-01-25T00:00:00.000Z":73,"2015-02-01T00:00:00.000Z":55}}

GET http://0.0.0.0/api/v1.0/skus/forecasts?wk_range=2015-01-01,2015-02-01&skus=1,4

{"1":{"2015-01-04T00:00:00.000Z":87,"2015-01-11T00:00:00.000Z":93,"2015-01-18T00:00:00.000Z":61,"2015-01-25T00:00:00.000Z":73,"2015-02-01T00:00:00.000Z":55},"4":{"2015-01-04T00:00:00.000Z":59,"2015-01-11T00:00:00.000Z":67,"2015-01-18T00:00:00.000Z":13,"2015-01-25T00:00:00.000Z":27,"2015-02-01T00:00:00.000Z":88}}

#### Note

Price version, pull price of each sku from its product information dictionary and multiply it to the output for it's unit counts to generate a dollar count.

Only setup for first 5 skus..

GET http://0.0.0.0/api/v1.0/skus/forecasts?wk_range=2017-10-01,2017-11-01&cat=cat1&price=True

{"0":{"2017-10-01T00:00:00.000Z":154.0,"2017-10-08T00:00:00.000Z":7.0,"2017-10-15T00:00:00.000Z":483.0,"2017-10-22T00:00:00.000Z":350.0,"2017-10-29T00:00:00.000Z":259.0},"1":{"2017-10-01T00:00:00.000Z":720.0,"2017-10-08T00:00:00.000Z":975.0,"2017-10-15T00:00:00.000Z":15.0,"2017-10-22T00:00:00.000Z":885.0,"2017-10-29T00:00:00.000Z":1275.0},"2":{"2017-10-01T00:00:00.000Z":40.5,"2017-10-08T00:00:00.000Z":436.5,"2017-10-15T00:00:00.000Z":90.0,"2017-10-22T00:00:00.000Z":189.0,"2017-10-29T00:00:00.000Z":256.5},"3":{"2017-10-01T00:00:00.000Z":35.94,"2017-10-08T00:00:00.000Z":533.11,"2017-10-15T00:00:00.000Z":131.78,"2017-10-22T00:00:00.000Z":419.3,"2017-10-29T00:00:00.000Z":329.45}}

# ii) 

Search skus based on date ranges, categories, or specific skus

GET http://0.0.0.0/api/v1.0/skus/forecasts?skus=1,4

{"1":{"2015-01-04T00:00:00.000Z":87,"2015-01-11T00:00:00.000Z":93,"2015-01-18T00:00:00.000Z":61,"2015-01-25T00:00:00.000Z":73,"2015-02-01T00:00:00.000Z":55,"2015-02-08T00:00:00.000Z":9,"2015-02-15T00:00:00.000Z":92,"2015-02-22T00:00:00.000Z":66,"2015-03-01T00:00:00.000Z":56,"2015-03-08T00:00:00.000Z":1,"2015-03-15T00:00:00.000Z":64,"2015-03-22T00:00:00.000Z":3,"2015-03-29T00:00:00.000Z":47,"2015-04-05T00:00:00.000Z":39,"2015-04-12T00:00:00.000Z":54,"2015-04-19T00:00:00.000Z":7,"2015-04-26T00:00:00.000Z":33,"2015-05-03T00:00:00.000Z":9,"2015-05-10T00:00:00.000Z":35,"2015-05-17T00:00:00.000Z":17,"2015-05-24T00:00:00.000Z":22,"2015-05-31T00:00:00.000Z":80,"2015-06-07T00:00:00.000Z":19,"2015-06-14T00:00:00.000Z":70,"2015-06-21T00:00:00.000Z":56,"2015-06-28T00:00:00.000Z":92,"2015-07-05T00:00:00.000Z":95,"2015-07-12T00:00:00.000Z":19,"2015-07-19T00:00:00.000Z":84,"2015-07-26T00:00:00.000Z":62,"2015-08-02T00:00:00.000Z":16,"2015-08-09T00:00:00.000Z":45,"2015-08-16T00:00:00.000Z":56,"2015-08-23T00:00:00.000Z":79,"2015-08-30T00:00:00.000Z":46,"2015-09-06T00:00:00.000Z":44,"2015-09-13T00:00:00.000Z":19,"2015-09-20T00:00:00.000Z":17,"2015-09-27T00:00:00.000Z":42,"2015-10-04T00:00:00.000Z":62,"2015-10-11T00:00:00.000Z":74,"2015-10-18T00:00:00.000Z":16,"2015-10-25T00:00:00.000Z":84,"2015-11-01T00:00:00.000Z":86,"2015-11-08T00:00:00.000Z":15,"2015-11-15T00:00:00.000Z":81,"2015-11-22T00:00:00.000Z":4,"2015-11-29T00:00:00.000Z":97,"2015-12-06T00:00:00.000Z":79,"2015-12-13T00:00:00.000Z":74,"2015-12-20T00:00:00.000Z":19,"2015-12-27T00:00:00.000Z":85,"2016-01-03T00:00:00.000Z":89,"2016-01-10T00:00:00.000Z":11,"2016-01-17T00:00:00.000Z":50,"2016-01-24T00:00:00.000Z":94,"2016-01-31T00:00:00.000Z":10,"2016-02-07T00:00:00.000Z":80,"2016-02-14T00:00:00.000Z":17,"2016-02-21T00:00:00.000Z":74,"2016-02-28T00:00:00.000Z":58,"2016-03-06T00:00:00.000Z":72,"2016-03-13T00:00:00.000Z":43,"2016-03-20T00:00:00.000Z":14,"2016-03-27T00:00:00.000Z":73,"2016-04-03T00:00:00.000Z":91,"2016-04-10T00:00:00.000Z":61,"2016-04-17T00:00:00.000Z":4,"2016-04-24T00:00:00.000Z":34,"2016-05-01T00:00:00.000Z":30,"2016-05-08T00:00:00.000Z":47,"2016-05-15T00:00:00.000Z":48,"2016-05-22T00:00:00.000Z":32,"2016-05-29T00:00:00.000Z":2,"2016-06-05T00:00:00.000Z":66,"2016-06-12T00:00:00.000Z":26,"2016-06-19T00:00:00.000Z":55,"2016-06-26T00:00:00.000Z":5,"2016-07-03T00:00:00.000Z":90,"2016-07-10T00:00:00.000Z":70,"2016-07-17T00:00:00.000Z":18,"2016-07-24T00:00:00.000Z":7,"2016-07-31T00:00:00.000Z":48,"2016-08-07T00:00:00.000Z":91,"2016-08-14T00:00:00.000Z":74,"2016-08-21T00:00:00.000Z":78,"2016-08-28T00:00:00.000Z":98,"2016-09-04T00:00:00.000Z":8,"2016-09-11T00:00:00.000Z":24,"2016-09-18T00:00:00.000Z":5,"2016-09-25T00:00:00.000Z":17,"2016-10-02T00:00:00.000Z":23,"2016-10-09T00:00:00.000Z":84,"2016-10-16T00:00:00.000Z":70,"2016-10-23T00:00:00.000Z":63,"2016-10-30T00:00:00.000Z":94,"2016-11-06T00:00:00.000Z":47,"2016-11-13T00:00:00.000Z":75,"2016-11-20T00:00:00.000Z":63,"2016-11-27T00:00:00.000Z":89,"2016-12-04T00:00:00.000Z":58,"2016-12-11T00:00:00.000Z":98,"2016-12-18T00:00:00.000Z":51,"2016-12-25T00:00:00.000Z":4,"2017-01-01T00:00:00.000Z":11,"2017-01-08T00:00:00.000Z":41,"2017-01-15T00:00:00.000Z":36,"2017-01-22T00:00:00.000Z":16,"2017-01-29T00:00:00.000Z":3,"2017-02-05T00:00:00.000Z":60,"2017-02-12T00:00:00.000Z":77,"2017-02-19T00:00:00.000Z":29,"2017-02-26T00:00:00.000Z":73,"2017-03-05T00:00:00.000Z":48,"2017-03-12T00:00:00.000Z":47,"2017-03-19T00:00:00.000Z":12,"2017-03-26T00:00:00.000Z":30,"2017-04-02T00:00:00.000Z":72,"2017-04-09T00:00:00.000Z":58,"2017-04-16T00:00:00.000Z":33,"2017-04-23T00:00:00.000Z":33,"2017-04-30T00:00:00.000Z":61,"2017-05-07T00:00:00.000Z":50,"2017-05-14T00:00:00.000Z":95,"2017-05-21T00:00:00.000Z":86,"2017-05-28T00:00:00.000Z":79,"2017-06-04T00:00:00.000Z":41,"2017-06-11T00:00:00.000Z":65,"2017-06-18T00:00:00.000Z":93,"2017-06-25T00:00:00.000Z":38,"2017-07-02T00:00:00.000Z":90,"2017-07-09T00:00:00.000Z":43,"2017-07-16T00:00:00.000Z":59,"2017-07-23T00:00:00.000Z":36,"2017-07-30T00:00:00.000Z":52,"2017-08-06T00:00:00.000Z":45,"2017-08-13T00:00:00.000Z":12,"2017-08-20T00:00:00.000Z":77,"2017-08-27T00:00:00.000Z":96,"2017-09-03T00:00:00.000Z":72,"2017-09-10T00:00:00.000Z":19,"2017-09-17T00:00:00.000Z":37,"2017-09-24T00:00:00.000Z":9,"2017-10-01T00:00:00.000Z":5,"2017-10-08T00:00:00.000Z":97,"2017-10-15T00:00:00.000Z":93,"2017-10-22T00:00:00.000Z":77,"2017-10-29T00:00:00.000Z":23,"2017-11-05T00:00:00.000Z":79,"2017-11-12T00:00:00.000Z":22,"2017-11-19T00:00:00.000Z":25,"2017-11-26T00:00:00.000Z":39},"4":{"2015-01-04T00:00:00.000Z":59,"2015-01-11T00:00:00.000Z":67,"2015-01-18T00:00:00.000Z":13,"2015-01-25T00:00:00.000Z":27,"2015-02-01T00:00:00.000Z":88,"2015-02-08T00:00:00.000Z":99,"2015-02-15T00:00:00.000Z":41,"2015-02-22T00:00:00.000Z":13,"2015-03-01T00:00:00.000Z":53,"2015-03-08T00:00:00.000Z":46,"2015-03-15T00:00:00.000Z":2,"2015-03-22T00:00:00.000Z":89,"2015-03-29T00:00:00.000Z":87,"2015-04-05T00:00:00.000Z":60,"2015-04-12T00:00:00.000Z":61,"2015-04-19T00:00:00.000Z":73,"2015-04-26T00:00:00.000Z":37,"2015-05-03T00:00:00.000Z":71,"2015-05-10T00:00:00.000Z":26,"2015-05-17T00:00:00.000Z":37,"2015-05-24T00:00:00.000Z":2,"2015-05-31T00:00:00.000Z":24,"2015-06-07T00:00:00.000Z":13,"2015-06-14T00:00:00.000Z":16,"2015-06-21T00:00:00.000Z":16,"2015-06-28T00:00:00.000Z":1,"2015-07-05T00:00:00.000Z":68,"2015-07-12T00:00:00.000Z":29,"2015-07-19T00:00:00.000Z":90,"2015-07-26T00:00:00.000Z":53,"2015-08-02T00:00:00.000Z":56,"2015-08-09T00:00:00.000Z":34,"2015-08-16T00:00:00.000Z":72,"2015-08-23T00:00:00.000Z":79,"2015-08-30T00:00:00.000Z":83,"2015-09-06T00:00:00.000Z":89,"2015-09-13T00:00:00.000Z":98,"2015-09-20T00:00:00.000Z":58,"2015-09-27T00:00:00.000Z":89,"2015-10-04T00:00:00.000Z":36,"2015-10-11T00:00:00.000Z":52,"2015-10-18T00:00:00.000Z":59,"2015-10-25T00:00:00.000Z":72,"2015-11-01T00:00:00.000Z":6,"2015-11-08T00:00:00.000Z":70,"2015-11-15T00:00:00.000Z":87,"2015-11-22T00:00:00.000Z":98,"2015-11-29T00:00:00.000Z":26,"2015-12-06T00:00:00.000Z":94,"2015-12-13T00:00:00.000Z":79,"2015-12-20T00:00:00.000Z":31,"2015-12-27T00:00:00.000Z":43,"2016-01-03T00:00:00.000Z":53,"2016-01-10T00:00:00.000Z":80,"2016-01-17T00:00:00.000Z":55,"2016-01-24T00:00:00.000Z":94,"2016-01-31T00:00:00.000Z":6,"2016-02-07T00:00:00.000Z":23,"2016-02-14T00:00:00.000Z":41,"2016-02-21T00:00:00.000Z":24,"2016-02-28T00:00:00.000Z":86,"2016-03-06T00:00:00.000Z":29,"2016-03-13T00:00:00.000Z":36,"2016-03-20T00:00:00.000Z":34,"2016-03-27T00:00:00.000Z":45,"2016-04-03T00:00:00.000Z":31,"2016-04-10T00:00:00.000Z":11,"2016-04-17T00:00:00.000Z":48,"2016-04-24T00:00:00.000Z":12,"2016-05-01T00:00:00.000Z":70,"2016-05-08T00:00:00.000Z":99,"2016-05-15T00:00:00.000Z":28,"2016-05-22T00:00:00.000Z":81,"2016-05-29T00:00:00.000Z":40,"2016-06-05T00:00:00.000Z":71,"2016-06-12T00:00:00.000Z":21,"2016-06-19T00:00:00.000Z":41,"2016-06-26T00:00:00.000Z":56,"2016-07-03T00:00:00.000Z":84,"2016-07-10T00:00:00.000Z":33,"2016-07-17T00:00:00.000Z":73,"2016-07-24T00:00:00.000Z":74,"2016-07-31T00:00:00.000Z":63,"2016-08-07T00:00:00.000Z":77,"2016-08-14T00:00:00.000Z":1,"2016-08-21T00:00:00.000Z":43,"2016-08-28T00:00:00.000Z":92,"2016-09-04T00:00:00.000Z":51,"2016-09-11T00:00:00.000Z":10,"2016-09-18T00:00:00.000Z":42,"2016-09-25T00:00:00.000Z":46,"2016-10-02T00:00:00.000Z":54,"2016-10-09T00:00:00.000Z":99,"2016-10-16T00:00:00.000Z":22,"2016-10-23T00:00:00.000Z":46,"2016-10-30T00:00:00.000Z":94,"2016-11-06T00:00:00.000Z":92,"2016-11-13T00:00:00.000Z":18,"2016-11-20T00:00:00.000Z":71,"2016-11-27T00:00:00.000Z":92,"2016-12-04T00:00:00.000Z":2,"2016-12-11T00:00:00.000Z":35,"2016-12-18T00:00:00.000Z":93,"2016-12-25T00:00:00.000Z":47,"2017-01-01T00:00:00.000Z":87,"2017-01-08T00:00:00.000Z":32,"2017-01-15T00:00:00.000Z":75,"2017-01-22T00:00:00.000Z":6,"2017-01-29T00:00:00.000Z":53,"2017-02-05T00:00:00.000Z":50,"2017-02-12T00:00:00.000Z":92,"2017-02-19T00:00:00.000Z":70,"2017-02-26T00:00:00.000Z":58,"2017-03-05T00:00:00.000Z":60,"2017-03-12T00:00:00.000Z":44,"2017-03-19T00:00:00.000Z":88,"2017-03-26T00:00:00.000Z":36,"2017-04-02T00:00:00.000Z":85,"2017-04-09T00:00:00.000Z":90,"2017-04-16T00:00:00.000Z":19,"2017-04-23T00:00:00.000Z":4,"2017-04-30T00:00:00.000Z":97,"2017-05-07T00:00:00.000Z":79,"2017-05-14T00:00:00.000Z":74,"2017-05-21T00:00:00.000Z":47,"2017-05-28T00:00:00.000Z":3,"2017-06-04T00:00:00.000Z":96,"2017-06-11T00:00:00.000Z":76,"2017-06-18T00:00:00.000Z":99,"2017-06-25T00:00:00.000Z":98,"2017-07-02T00:00:00.000Z":51,"2017-07-09T00:00:00.000Z":10,"2017-07-16T00:00:00.000Z":27,"2017-07-23T00:00:00.000Z":3,"2017-07-30T00:00:00.000Z":56,"2017-08-06T00:00:00.000Z":59,"2017-08-13T00:00:00.000Z":84,"2017-08-20T00:00:00.000Z":0,"2017-08-27T00:00:00.000Z":80,"2017-09-03T00:00:00.000Z":51,"2017-09-10T00:00:00.000Z":53,"2017-09-17T00:00:00.000Z":47,"2017-09-24T00:00:00.000Z":4,"2017-10-01T00:00:00.000Z":1,"2017-10-08T00:00:00.000Z":2,"2017-10-15T00:00:00.000Z":58,"2017-10-22T00:00:00.000Z":39,"2017-10-29T00:00:00.000Z":44,"2017-11-05T00:00:00.000Z":78,"2017-11-12T00:00:00.000Z":66,"2017-11-19T00:00:00.000Z":71,"2017-11-26T00:00:00.000Z":14}}

#### Note

Original idea of format:

In [ ]:
{"Forecasts":
    [
        {
        "SKU":"123456",
        "Forecast":
            {
                "01012017":"100",
                "01082017":"97",
                "01152017":"80",
                "01222017":"96",
                "01292017":"103"
            },
        },
        {
        "SKU":"234567",
        "Forecast":
            {
                "01012017":"23",
                "01082017":"34",
                "01152017":"32",
                "01222017":"32",
                "01292017":"29"
            }
        },
        ...
        {
        "SKU":"345678",
        "Forecast":
            {
                "01012017":"2",
                "01082017":"2",
                "01152017":"2",
                "01222017":"4",
                "01292017":"3"
            }
        },
    ]
}

GET http://0.0.0.0/api/v1.0/skus/forecasts?wk_range=2015-01-01,2015-02-01&cat=cat3

{"7":{"2015-01-04T00:00:00.000Z":49,"2015-01-11T00:00:00.000Z":10,"2015-01-18T00:00:00.000Z":34,"2015-01-25T00:00:00.000Z":42,"2015-02-01T00:00:00.000Z":82},"8":{"2015-01-04T00:00:00.000Z":37,"2015-01-11T00:00:00.000Z":21,"2015-01-18T00:00:00.000Z":55,"2015-01-25T00:00:00.000Z":56,"2015-02-01T00:00:00.000Z":80},"9":{"2015-01-04T00:00:00.000Z":48,"2015-01-11T00:00:00.000Z":83,"2015-01-18T00:00:00.000Z":64,"2015-01-25T00:00:00.000Z":59,"2015-02-01T00:00:00.000Z":98}}

# iii)

Override values for specific sku for given date, can do multiple at once

override forecast for specific week/sku:

curl -d '{"skus":{"0":{"2017-07-23": "0","2017-07-30": "0"}}}' -H "Content-Type: application/json" -X POST http://0.0.0.0/api/v1.0/skus/forecasts

Example format:

POST

{
    "skus": {
        "0": {
            "2015-01-04": "11",
            "2015-01-11": "25"
        },
        "1": {
            "2015-01-04": "2",
            "2015-01-11": "4"
        }
    }
}

#### Note

I don't understand how updating a total would work without modifying the underlying data which doesn't make sense to me..

My best guess would be to take the total over all weeks they want to update, take the difference between the old total and new total, and then divide it by the number of weeks the total is for, and then add that value to each week's forecast. But because this seems like a stretch in logic - I would ask the requester for more information before making modifications like that.

----

Part 2) 

Suppose you were to deploy this service. 
    - What resources would you need to host this service? 
    - What tools would you use for monitoring the health of the service? 
    - What tools would you use for source code management, testing, deployment, and automation? 
    - How would you configure a continuous delivery pipeline using these tools? 
    - Which of these tools have you used in prior projects?

#### Note

1. A server hosted by company or whatever service you currently use to host apis like amazon api gateway or google api engine. 
2. I used postman to send some POST requests and looked into its other options. You can schedule monitoring of the API with tests and other timed events.
3. I would use github for source code management, branch for different versions, and use autodeployment when commiting new changes to the codebase. 
4. Perhaps Gitlab or some other platform. But I am not clear on what CI/CD is.
5. I've used github plenty before but I have not used the autodeployment feature. I also have used postman for POST requests but not for monitoring health of a service. 


-----

Part 3) 

Implement a web service that consumes the requests and produces the responses you scoped out in the first exercise. For the sake of this exercise, you may limit service to only provide forecasts for 10 SKUs. You may use dummy values for the forecasts and SKU numbers.

Manually overridden forecasts should be recorded and contained in subsequent requests to the service. Moreover, forecasts should be automatically updated to remain consistent with manual overrides to the total forecasted demand across all weeks or all SKUs. 

Once completed, please provide an endpoint for your web service.

#### Note

run app with command: 'sudo python hello.py'

access the api at url: 'http://0.0.0.0/api/v1.0/skus/forecasts'


request:
http://0.0.0.0/api/v1.0/skus/forecasts?wk_range=2017-07-23,2017-10-01&skus=1,3

response:
{"1":{"2017-07-23T00:00:00.000Z":2,"2017-07-30T00:00:00.000Z":4,"2017-08-06T00:00:00.000Z":22,"2017-08-13T00:00:00.000Z":51,"2017-08-20T00:00:00.000Z":101,"2017-08-27T00:00:00.000Z":9,"2017-09-03T00:00:00.000Z":45,"2017-09-10T00:00:00.000Z":21,"2017-09-17T00:00:00.000Z":67,"2017-09-24T00:00:00.000Z":79,"2017-10-01T00:00:00.000Z":48},"3":{"2017-07-23T00:00:00.000Z":79,"2017-07-30T00:00:00.000Z":34,"2017-08-06T00:00:00.000Z":58,"2017-08-13T00:00:00.000Z":87,"2017-08-20T00:00:00.000Z":70,"2017-08-27T00:00:00.000Z":77,"2017-09-03T00:00:00.000Z":73,"2017-09-10T00:00:00.000Z":45,"2017-09-17T00:00:00.000Z":72,"2017-09-24T00:00:00.000Z":87,"2017-10-01T00:00:00.000Z":6}}

request:
http://0.0.0.0/api/v1.0/skus/forecasts?wk_range=2017-07-23&skus=1,3

response:
"1":{"2017-07-23T00:00:00.000Z":2},"3":{"2017-07-23T00:00:00.000Z":79}}

request:
http://0.0.0.0/api/v1.0/skus/forecasts?wk_range=2017-10-01&cat=cat2

response:
{"4":{"2017-10-01T00:00:00.000Z":3},"5":{"2017-10-01T00:00:00.000Z":58},"6":{"2017-10-01T00:00:00.000Z":27}}

request:
http://0.0.0.0/api/v1.0/skus/forecasts?wk_range=2017-10-01,2017-11-01&skus=1,4,9&total=sku

response:
{"1":258,"4":166,"9":195}

request:
http://0.0.0.0/api/v1.0/skus/forecasts?wk_range=2017-10-01,2017-11-01&cat=cat1&total=grand

response:
{
  "grand total": "904"
}

---

Part 4) 

Implement a web application that provides an interface to your service. Your application’s interface should contain a table similar to the one shown below:

For 10 fictional SKUs above table contains:
    - Actual demand for last 3 weeks (wk1-wk3)
    - Demand forecasts for 10 consecutive weeks (wk4-wk13, assuming current week is wk4 of Q1)

Additionally, the table contains totals of demand projections across weeks for each SKU, totals for SKUs for each week, and grand total for entire category and quarter. This table should be populated by making a call to your web service. Users should be allowed to edit values in appropriate cells (to be determined by you) which should result in a manual override call to the forecast service, as well as a call to the service to repopulate the table with
revised forecasts (please come up with appropriate rules to derive SKU-Wk level overridden forecast using the values provided by user in editable cells). 

Once completed, please provide a URL for your web application.

#### Note

Table url: http://0.0.0.0/api/v1.0/skus/forecasts/table

I've made the future and current week forecasts of data editable to the end user. The rest of the data is frozen in the GUI since it makes sense to not modify the past data points and the totals - which are calculated each run based on the values of the forecasts.

In addition, you can filter the table as you can with the forecast GET requests above. By category and individual sku. The date range will always center around the current date.

---

Part 5) 

- What mechanisms/rules should be put in place to prevent conflicting output for multiple end-users? 
- How should distributed computing be engineered to scale the application for 1M SKUs, 52 weeks, and 1K concurrent end-users with service response time SLA (P99) of 100ms ?

#### Note

1. I'm not familiar with much of the software engineering aspects of building a producion ready API so these unfortunately are things I've never run into before. I know my setup would need to be modified to be more robust by using a SQL database to store SKU data on the backend. But in terms of how to modify the setup to distribute to end-users I would take advice from a more knowledgable engineer.

I do understand that SLA (P99) measn that 99% of all responses would need to be under 100ms. 

# Sketch Pad

In [475]:
import numpy as np
import pandas as pd

dates = pd.date_range('2015-01', '2017-12', freq='W',format='%Y%m%d')
data = pd.DataFrame(np.random.randint(100, size=(len(dates), 10)))
data.set_index(dates, inplace=True)
new_cols = []
for i in data.columns:
    new_cols.append(str(i))
data.columns = new_cols

In [484]:
data.loc[wk_start:wk_end][skus].dt.strftime('%m/%d/%Y').to_json(date_format='iso')

AttributeError: 'DatetimeIndex' object has no attribute 'dt'

In [215]:
data.loc['2015-01-04':'']

0    41
1    46
2    19
3     4
4    36
5     5
6    72
7    86
8    69
9    17
Name: 2015-01-04 00:00:00, dtype: int64

In [210]:
str_dates = []
for date in dates.map(pd.Timestamp.date):
    str_dates.append(str(date))

In [211]:
str_dates

['2015-01-04',
 '2015-01-11',
 '2015-01-18',
 '2015-01-25',
 '2015-02-01',
 '2015-02-08',
 '2015-02-15',
 '2015-02-22',
 '2015-03-01',
 '2015-03-08',
 '2015-03-15',
 '2015-03-22',
 '2015-03-29',
 '2015-04-05',
 '2015-04-12',
 '2015-04-19',
 '2015-04-26',
 '2015-05-03',
 '2015-05-10',
 '2015-05-17',
 '2015-05-24',
 '2015-05-31',
 '2015-06-07',
 '2015-06-14',
 '2015-06-21',
 '2015-06-28',
 '2015-07-05',
 '2015-07-12',
 '2015-07-19',
 '2015-07-26',
 '2015-08-02',
 '2015-08-09',
 '2015-08-16',
 '2015-08-23',
 '2015-08-30',
 '2015-09-06',
 '2015-09-13',
 '2015-09-20',
 '2015-09-27',
 '2015-10-04',
 '2015-10-11',
 '2015-10-18',
 '2015-10-25',
 '2015-11-01',
 '2015-11-08',
 '2015-11-15',
 '2015-11-22',
 '2015-11-29',
 '2015-12-06',
 '2015-12-13',
 '2015-12-20',
 '2015-12-27',
 '2016-01-03',
 '2016-01-10',
 '2016-01-17',
 '2016-01-24',
 '2016-01-31',
 '2016-02-07',
 '2016-02-14',
 '2016-02-21',
 '2016-02-28',
 '2016-03-06',
 '2016-03-13',
 '2016-03-20',
 '2016-03-27',
 '2016-04-03',
 '2016-04-

In [184]:
data['1'].head()

2015-01-04    65
2015-01-11    14
2015-01-18    35
2015-01-25    38
2015-02-01    55
Freq: W-SUN, Name: 1, dtype: int64

In [219]:
wk_range = '2015-01-04,2015-01-25'
#wk_range = '2015-01-04'
#wk_range = ''
#skus = [0,1]
skus = ['1','2']

In [220]:
if ',' in wk_range:
    wk_start, wk_end = wk_range.split(',')
else:
    wk_start = wk_range
    wk_end = wk_range

In [222]:
if wk_range:
    print(data.loc[wk_start:wk_end][skus])
else:
    print(data.iloc[skus])

             1   2
2015-01-04  46  19
2015-01-11  93  24
2015-01-18  38  20
2015-01-25  52  84


In [249]:
if wk_range:
    print(data.loc[wk_start:wk_end][skus].sum())
else:
    print(data.iloc[skus].sum())

1    229
2    147
3    193
4    245
5    139
dtype: int64


In [238]:
#format for wk_range and skus

data.loc[wk_start:wk_end].to_json()

#data.loc['2015-01-05':'2015-01-25'][0]

'{"0":{"1420329600000":41,"1420934400000":26,"1421539200000":20,"1422144000000":83},"1":{"1420329600000":46,"1420934400000":93,"1421539200000":38,"1422144000000":52},"2":{"1420329600000":19,"1420934400000":24,"1421539200000":20,"1422144000000":84},"3":{"1420329600000":4,"1420934400000":99,"1421539200000":78,"1422144000000":12},"4":{"1420329600000":36,"1420934400000":85,"1421539200000":45,"1422144000000":79},"5":{"1420329600000":5,"1420934400000":23,"1421539200000":91,"1422144000000":20},"6":{"1420329600000":72,"1420934400000":41,"1421539200000":18,"1422144000000":95},"7":{"1420329600000":86,"1420934400000":2,"1421539200000":23,"1422144000000":95},"8":{"1420329600000":69,"1420934400000":70,"1421539200000":20,"1422144000000":71},"9":{"1420329600000":17,"1420934400000":83,"1421539200000":31,"1422144000000":0}}'

In [84]:
data.loc[wk_range].to_dict()

{0: 22, 1: 91, 2: 23, 3: 63, 4: 11, 5: 60, 6: 51, 7: 78, 8: 10, 9: 79}

In [240]:
skus = '1,2,3,4,5' 

In [241]:
skus = skus.split(',')

In [242]:
num_skus = []

for sku in skus:
    num_skus.append(int(sku))

In [248]:
data['1'].sum()

7772

In [292]:
override = {
    "skus": {
        "0": {
            "2015-01-04": "15",
            "2015-01-11": "32"
        }
    }
}

In [ ]:
{"skus":{"0":{"2015-01-04": "15","2015-01-11": "32"}}}

In [ ]:
curl -d '{"skus":{"0":{"2017-07-23": "0","2017-07-30": "0"}}}' -H "Content-Type: application/json" -X POST http://0.0.0.0/api/v1.0/skus/forecasts
        
        

In [ ]:
curl -H "Content-Type: application/json" -X POST -d '{"username":"xyz","password":"xyz"}' http://localhost:3000/api/login
        
        
        

In [293]:
for sku in override['skus'].keys():
    for date,val in override['skus'][sku].items():
        #print(date,val)
        data.loc[date][sku] = val

In [295]:
data['0'].head()

2015-01-04    15
2015-01-11    32
2015-01-18    20
2015-01-25    83
2015-02-01    93
Freq: W-SUN, Name: 0, dtype: int64

In [267]:
for k,v in override['skus']['0'].keys()

dict_keys(['2015-01-04', '2015-01-11'])

In [296]:
cats = {'cat1':['0','1','2','3'],'cat2':['4','5','6'],'cat3':['7','8','9']}

In [297]:
cats['cat1']

['0', '1', '2', '3']

In [317]:
data.loc[wk_start:wk_end][skus].T.to_dict(orient='split')#.items()

{'columns': [Timestamp('2015-01-04 00:00:00', freq='W-SUN'),
  Timestamp('2015-01-11 00:00:00', freq='W-SUN'),
  Timestamp('2015-01-18 00:00:00', freq='W-SUN'),
  Timestamp('2015-01-25 00:00:00', freq='W-SUN')],
 'data': [[46, 93, 38, 52],
  [19, 24, 20, 84],
  [4, 99, 78, 12],
  [36, 85, 45, 79],
  [5, 23, 91, 20]],
 'index': ['1', '2', '3', '4', '5']}

In [318]:
data.loc[wk_start:wk_end][skus].T.to_dict(orient='split')['columns']

[Timestamp('2015-01-04 00:00:00', freq='W-SUN'),
 Timestamp('2015-01-11 00:00:00', freq='W-SUN'),
 Timestamp('2015-01-18 00:00:00', freq='W-SUN'),
 Timestamp('2015-01-25 00:00:00', freq='W-SUN')]

In [320]:
data.loc[wk_start:wk_end][skus].T.to_dict(orient='split')['index']

['1', '2', '3', '4', '5']

In [328]:
for i,j in data.loc[wk_start:wk_end][skus].T.to_dict(orient='split').items():
    print(j)

['1', '2', '3', '4', '5']
[Timestamp('2015-01-04 00:00:00', freq='W-SUN'), Timestamp('2015-01-11 00:00:00', freq='W-SUN'), Timestamp('2015-01-18 00:00:00', freq='W-SUN'), Timestamp('2015-01-25 00:00:00', freq='W-SUN')]
[[46, 93, 38, 52], [19, 24, 20, 84], [4, 99, 78, 12], [36, 85, 45, 79], [5, 23, 91, 20]]


In [474]:
data.loc[wk_start:wk_end][skus]

TypeError: '>' not supported between instances of 'str' and 'datetime.date'

In [340]:
filter_df = data.loc[wk_start:wk_end][skus]

In [397]:
wk_totals = data.loc[wk_start:wk_end][skus].sum(axis=1)
filter_df = pd.concat([filter_df,wk_totals.rename('Total')],axis=1)
filter_df = filter_df.append(filter_df.sum(numeric_only=True).rename('Total'))

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [394]:
filter_df

,1,2,3,4,5,Total
2015-01-04 00:00:00,46,19,4,36,5,110
2015-01-11 00:00:00,93,24,99,85,23,324
2015-01-18 00:00:00,38,20,78,45,91,272
2015-01-25 00:00:00,52,84,12,79,20,247
Total,229,147,193,245,139,953


In [393]:
filter_df.T.to_dict(orient='split').items()

dict_items([('index', ['1', '2', '3', '4', '5', 'Total']), ('columns', [Timestamp('2015-01-04 00:00:00', freq='W-SUN'), Timestamp('2015-01-11 00:00:00', freq='W-SUN'), Timestamp('2015-01-18 00:00:00', freq='W-SUN'), Timestamp('2015-01-25 00:00:00', freq='W-SUN'), 'Total']), ('data', [[46, 93, 38, 52, 229], [19, 24, 20, 84, 147], [4, 99, 78, 12, 193], [36, 85, 45, 79, 245], [5, 23, 91, 20, 139], [110, 324, 272, 247, 953]])])

In [429]:
dates

DatetimeIndex(['2017-07-23', '2017-07-30', '2017-08-06', '2017-08-13',
               '2017-08-20', '2017-08-27', '2017-09-03', '2017-09-10',
               '2017-09-17', '2017-09-24', '2017-10-01', '2017-10-08',
               '2017-10-15'],
              dtype='datetime64[ns]', freq='W-SUN')

In [413]:
import datetime as dt

In [414]:
date = dt.date.today()

In [425]:
from datetime import datetime, timedelta

wk_start = (datetime.today() - timedelta(days=21)).date()
wk_end = (datetime.today() + timedelta(days=63)).date()

In [426]:
wk_end

datetime.date(2017, 10, 19)

In [460]:
dates = pd.date_range('2017-07-23',periods=50, freq='W')
data = pd.DataFrame(np.random.randint(100, size=(len(dates), 10)))
data.set_index(dates, inplace=True)
new_cols = []
for i in data.columns:
    new_cols.append(str(i))
data.columns = new_cols

In [461]:
data.to_csv('data.csv')

In [462]:
data = pd.read_csv('data.csv',index_col=0)

In [471]:
data.T.to_dict(orient='split')

{'columns': ['2017-07-23',
  '2017-07-30',
  '2017-08-06',
  '2017-08-13',
  '2017-08-20',
  '2017-08-27',
  '2017-09-03',
  '2017-09-10',
  '2017-09-17',
  '2017-09-24',
  '2017-10-01',
  '2017-10-08',
  '2017-10-15',
  '2017-10-22',
  '2017-10-29',
  '2017-11-05',
  '2017-11-12',
  '2017-11-19',
  '2017-11-26',
  '2017-12-03',
  '2017-12-10',
  '2017-12-17',
  '2017-12-24',
  '2017-12-31',
  '2018-01-07',
  '2018-01-14',
  '2018-01-21',
  '2018-01-28',
  '2018-02-04',
  '2018-02-11',
  '2018-02-18',
  '2018-02-25',
  '2018-03-04',
  '2018-03-11',
  '2018-03-18',
  '2018-03-25',
  '2018-04-01',
  '2018-04-08',
  '2018-04-15',
  '2018-04-22',
  '2018-04-29',
  '2018-05-06',
  '2018-05-13',
  '2018-05-20',
  '2018-05-27',
  '2018-06-03',
  '2018-06-10',
  '2018-06-17',
  '2018-06-24',
  '2018-07-01'],
 'data': [[54,
   54,
   72,
   71,
   85,
   98,
   70,
   91,
   35,
   60,
   18,
   25,
   57,
   64,
   68,
   21,
   33,
   89,
   78,
   44,
   81,
   13,
   87,
   67,
   18,
   77

In [516]:
skus = ['1','2','3']

In [517]:
prod_info = {
        "Product Information":
        {
        "0":{
        "SKU":"0",
        "Product Description":"Lined Paper, 50 pages",
        "Price":"7.00",
        "Promotion":"6.50",
        "Product Category":"paper_goods"
        },
        "1":{
        "SKU":"1",
        "Product Description":"Legal Paper, 50 pages",
        "Price":"15.00",
        "Promotion":"13.00",
        "Product Category":"paper_goods"
        },
        "2":{
        "SKU":"2",
        "Product Description":"Red Stapler",
        "Price":"4.50",
        "Promotion":"nan",
        "Product Category":"fasteners"
        },
        "3":{
        "SKU":"3",
        "Product Description":"Heavy Duty Stapler",
        "Price":"5.99",
        "Promotion":"nan",
        "Product Category":"fasteners"
        },
        "4":{
        "SKU":"4",
        "Product Description":"Paper Clips (1000ct)",
        "Price":"9.99",
        "Promotion":"nan",
        "Product Category":"fasteners"
        }
        }
        }

In [518]:
prices = []
for sku in skus:
    prices.append(float(prod_info['Product Information'][sku]['Price']))

In [519]:
prices

[15.0, 4.5, 5.99]

In [520]:
data.loc[wk_start:wk_end][skus]*prices#.sum().sum()

,1,2,3
2017-07-30,405.0,99.0,113.81
2017-08-06,1260.0,346.5,179.70
2017-08-13,165.0,184.5,113.81
2017-08-20,1170.0,342.0,473.21
2017-08-27,240.0,31.5,185.69
2017-09-03,270.0,261.0,545.09
2017-09-10,1290.0,148.5,167.72
2017-09-17,1140.0,81.0,251.58
2017-09-24,450.0,162.0,401.33
2017-10-01,1350.0,225.0,317.47
